In [1]:
import requests
import pandas as pd
import time
from pygame import mixer #for sound
from datetime import datetime, timedelta #for looping over date
import numpy as np

pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
API_key = 'JTFthLtGYuKFQK47MPvQcrabXJRJPk5nUEgaric4f17oP5VJcHyHqsqCMWXu'

#append to dictipnary 
def app_dictionary(id, list, dictionary):
    if id not in dictionary: #checks if column with key already exists 
        dictionary[id] = [list]
    else:
        dictionary[id].append(list) #if yes, it appends to key value

#running api calls and returning result + pagination
def run_api(api_call):
    try:
        result = requests.get(api_call)
        result = result.json()
    except ValueError:
        print(f"Failed to decode JSON. Please check API call: {api_call}")

    #checks if there is a next page
    #if yes it returns true, if no it returns false
    try:
        has_more = result['pagination']['has_more']
    except:
        has_more = False

    try:
        rate_limit = result['rate_limit']['remaining']
        reset = result['rate_limit']['resets_in_seconds']
    except:
        print('no rate limit available')

    return result, has_more, rate_limit, reset

#function for playing sound
def play_sound():
    mixer.init()
    mixer.music.load('done-for-you.mp3')
    mixer.music.play()

def run_pages(api_first, api_second): 
    results, has_more, rate_limit, reset = run_api(f"{api_first}{api_second}")

    page = 2
    while has_more == True:
        if int(rate_limit) > 1:
            next_results, has_more, rate_limit, reset = run_api(f"{api_first}page={page}&{api_second}")
            results['data'] = results['data'] + next_results['data']
            page += 1
        else:
            print(f'Starting sleep timer for: {reset} seconds...')
            time.sleep(int(reset))
    else:
        pass
        
    return results

### **Load data**
<br>
Need to find which teams have played for which seasons, and then set the time interval to the season duration and only get the data for the correct teams.

In [3]:
teams_df = pd.read_csv('data/teams.csv')
seasons_df = pd.read_csv('data/seasons.csv')
leagues_df = pd.read_csv('data/leagues.csv')

Creating dictionary with seasons start date, end date, id, and teams participating in the season

In [48]:
season_dict = {'id':[], 'starting_at':[], 'ending_at':[]}
for row in seasons_df.index:
    app_dictionary('id', str(seasons_df.iloc[row]['id']), season_dict)
    app_dictionary('starting_at', str(seasons_df.iloc[row]['starting_at']), season_dict)
    app_dictionary('ending_at', seasons_df.iloc[row]['ending_at'], season_dict)
    app_dictionary('teams', [i for i in teams_df.loc[teams_df['season_id'] == int(seasons_df.iloc[row]['id'])]['id']], season_dict)


In [164]:
season_dict = pd.DataFrame.from_dict(season_dict)
season_dict.to_csv('data/seasons_info.csv')
season_dict.head()

,id,starting_at,ending_at,teams
0,19734,2022-08-05,2023-05-28,"[78, 11, 8, 19, 42, 29, 71, 51, 14, 236, 52, 2..."
1,19744,2022-08-05,2023-05-27,"[3317, 1079, 510, 3321, 3543, 68, 82, 3320, 67..."
2,19799,2022-08-12,2023-06-04,"[3477, 3468, 485, 36, 1099, 361, 6827, 645, 23..."
3,19806,2022-08-13,2023-06-11,"[10722, 2930, 397, 522, 1123, 625, 345, 1628, ..."
4,21646,2023-08-11,2024-05-19,"[78, 11, 8, 19, 29, 115, 51, 21, 14, 236, 52, ..."


In [180]:
start_dates = season_dict['starting_at']
end_dates = season_dict['ending_at']
teams_inSeasons = season_dict['teams']
season_id = season_dict['id']

# This allows you to work with all rows at once
for season_start, season_end, teams_inSeason, season_id in zip(start_dates, end_dates, teams_inSeasons, season_id):
    print(f'Starting: {season_start}, {season_end}, {teams_inSeason}, {season_id}')
    fixtures = {'fixture_id':[], 'name': [], 'league_id':[], 'season_id':[], 'venue_id':[], 'formation_away':[], 'formation_home': [], 'referee':[]}
    events = []
    weather_pitch = {'fixture_id':[], 'pitch': [], 'weatherDescription':[], 'current':[]}
    for team in teams_inSeason:
        start_date = datetime.strptime(season_start, '%Y-%m-%d')
        end_date = datetime.strptime(season_end, '%Y-%m-%d')
        #find parameters
        total_days = (end_date - start_date).days
        days_interval = 100

        # Variable to store the previous date
        previous_date = None
        # Loop over the 300 days in 100-day intervals\n",
        for i in range(0, total_days, days_interval):
            current_date = start_date + timedelta(days=i)
            formatted_date = current_date.strftime('%Y-%m-%d')

            # Print the previous and current date,
            if previous_date:
                print(f"Previous date: {previous_date} -> Next date: {formatted_date}")
                first = 'https://api.sportmonks.com/v3/football/fixtures/between/' + str(previous_date) + '/' + str(formatted_date) + '/' + str(team) + '?'
                second = 'api_token=' + API_key + '&per_page=50&select=name&include=metadata;formations;weatherReport:description,current;referees;events:type_id,addition,minute&filters=eventTypes:10,19,20,21;metadataTypes:567' 
                results = run_pages(first, second)
                
                
                for result in results['data']:
                    #all events
                    events = events + result['events']

                    #creating weather dataset
                    weather_pitch['fixture_id'].append(result['id'])
                    if result['metadata']:
                        weather_pitch['pitch'].append(result['metadata'][0]['values']['pitch'])
                    else:
                        weather_pitch['pitch'].append('NaN')
                    try:
                        weather_pitch['weatherDescription'].append(result['weatherreport']['description'])
                    except TypeError:
                        weather_pitch['weatherDescription'].append('NaN')
                    try:
                        weather_pitch['current'].append(result['weatherreport']['current'])
                    except TypeError:
                        weather_pitch['current'].append('NaN')

                    #creating fixture dataset
                    fixtures['fixture_id'].append(result['id'])
                    fixtures['name'].append(result['name'])
                    fixtures['league_id'].append(result['league_id'])
                    fixtures['season_id'].append(result['season_id'])
                    fixtures['venue_id'].append(result['venue_id'])
                    fixtures['formation_away'].append(result['formations'][0]['formation'])
                    fixtures['formation_home'].append(result['formations'][1]['formation'])
                    #choosing the main referee -> type 6
                    try:
                        fixtures['referee'].append([referee['id'] for referee in result['referees'] if referee['type_id'] == 6][0])
                    except IndexError:
                         fixtures['referee'].append(result['referees'])
                
                 
            else: # Avoiding calling only the first date 
                print(f"First date: {formatted_date}") 
        
            previous_date = formatted_date

    #saving dataframe
    events = {'data': events}
    events = pd.DataFrame.from_dict(events['data'])
    fixtures = pd.DataFrame.from_dict(fixtures)
    weather_pitch = pd.DataFrame.from_dict(weather_pitch)

    #dropping duplicates 
    events.drop_duplicates()
    fixtures.drop_duplicates(subset=['fixture_id'], inplace=True)
    weather_pitch.drop_duplicates(subset=['fixture_id'], inplace=True)

    #saving as csv
    events.to_csv(f'data/events/events_{season_id}.csv')
    fixtures.to_csv(f'data/fixtures/fixtures_{season_id}.csv') 
    weather_pitch.to_csv(f'data/weather/weather_{season_id}.csv')


Starting: 2022-08-05, 2023-05-28, [78, 11, 8, 19, 42, 29, 71, 51, 14, 236, 52, 20, 63, 13, 9, 1, 65, 18, 15, 6], 19734
First date: 2022-08-05
Previous date: 2022-08-05 -> Next date: 2022-11-13
Previous date: 2022-11-13 -> Next date: 2023-02-21
First date: 2022-08-05
Previous date: 2022-08-05 -> Next date: 2022-11-13
Previous date: 2022-11-13 -> Next date: 2023-02-21
First date: 2022-08-05
Previous date: 2022-08-05 -> Next date: 2022-11-13
Previous date: 2022-11-13 -> Next date: 2023-02-21
First date: 2022-08-05
Previous date: 2022-08-05 -> Next date: 2022-11-13
Previous date: 2022-11-13 -> Next date: 2023-02-21
First date: 2022-08-05
Previous date: 2022-08-05 -> Next date: 2022-11-13
Previous date: 2022-11-13 -> Next date: 2023-02-21
First date: 2022-08-05
Previous date: 2022-08-05 -> Next date: 2022-11-13
Previous date: 2022-11-13 -> Next date: 2023-02-21
First date: 2022-08-05
Previous date: 2022-08-05 -> Next date: 2022-11-13
Previous date: 2022-11-13 -> Next date: 2023-02-21
First 

IndexError: list index out of range

In [144]:
first = 'https://api.sportmonks.com/v3/football/fixtures/between/2022-08-05/2022-09-10/78?'
second = 'api_token=' + API_key + '&per_page=50&select=name&include=metadata;formations;weatherReport:description,current;referees;events:type_id,addition,minute&filters=eventTypes:10,19,20,21;metadataTypes:567'
results = run_pages(first,second)
results

{'data': [{'name': 'Manchester United vs Brighton & Hove Albion',
   'id': 18535049,
   'sport_id': 1,
   'round_id': 274668,
   'stage_id': 77457864,
   'group_id': None,
   'aggregate_id': None,
   'league_id': 8,
   'season_id': 19734,
   'venue_id': 206,
   'state_id': 5,
   'starting_at_timestamp': None,
   'metadata': [{'id': 1157793,
     'metadatable_id': 18535049,
     'type_id': 567,
     'value_type': 'object',
     'values': {'pitch': 'GOOD'}}],
   'formations': [{'id': 336844,
     'fixture_id': 18535049,
     'participant_id': 78,
     'formation': '3-4-2-1',
     'location': 'away'},
    {'id': 336843,
     'fixture_id': 18535049,
     'participant_id': 14,
     'formation': '4-2-3-1',
     'location': 'home'}],
   'weatherreport': {'id': 103325,
    'fixture_id': 18535049,
    'venue_id': 206,
    'description': 'broken clouds',
    'current': None},
   'referees': [{'id': 3014813,
     'fixture_id': 18535049,
     'referee_id': 13537,
     'type_id': 9},
    {'id': 301

In [176]:
fixtures = {'fixture_id':[], 'name': [], 'league_id':[], 'season_id':[], 'venue_id':[], 'formation_away':[], 'formation_home': [], 'referee':[]}
events = []
weather_pitch = {'fixture_id':[], 'pitch': [], 'weatherDescription':[], 'current':[]}
referee = {}
for result in results['data']:
    #all events
    events = events + result['events']

    #creating weather dataset
    weather_pitch['fixture_id'].append(result['id'])
    if result['metadata']:
        weather_pitch['pitch'].append(result['metadata'][0]['values']['pitch'])
    else:
        weather_pitch['pitch'].append('NaN')
    if result.get('weatherreport') and 'description' in result['weatherreport']:
        weather_pitch['weatherDescription'].append(result['weatherreport']['description'])
    else:
        weather_pitch['weatherDescription'].append('NaN')
    if result.get('weatherreport') and 'description' in result['weatherreport']:
        weather_pitch['current'].append(result['weatherreport']['current'])
    else:
        weather_pitch['current'].append('NaN')
    
    #creating fixture dataset
    fixtures['fixture_id'].append(result['id'])
    fixtures['name'].append(result['name'])
    fixtures['league_id'].append(result['league_id'])
    fixtures['season_id'].append(result['season_id'])
    fixtures['venue_id'].append(result['venue_id'])
    fixtures['formation_away'].append(result['formations'][0]['formation'])
    fixtures['formation_home'].append(result['formations'][1]['formation'])
    #choosing the main referee -> type 6
    fixtures['referee'].append(result['referees'])


In [126]:
fixtures = pd.DataFrame.from_dict(fixtures)

In [134]:
events = {'data': events}
events = pd.DataFrame.from_dict(events['data'])
events.to_csv('data/events.csv')

In [177]:
weather_pitch

{'fixture_id': [18535041,
  18535056,
  18535242,
  18645567,
  18535257,
  18535067,
  18535077,
  18535093,
  18535105,
  18535115,
  18535277,
  18535283,
  18535125,
  18535139,
  18671647,
  18535297],
 'pitch': ['GOOD',
  'GOOD',
  'GOOD',
  'NaN',
  'GOOD',
  'GOOD',
  'GOOD',
  'GOOD',
  'GOOD',
  'GOOD',
  'GOOD',
  'GOOD',
  'GOOD',
  'GOOD',
  'NaN',
  'GOOD'],
 'weatherDescription': ['sky is clear',
  'light rain',
  'scattered clouds',
  'overcast clouds',
  'light rain',
  'overcast clouds',
  'NaN',
  'light rain',
  'few clouds',
  'NaN',
  'sky is clear',
  'light rain',
  'light rain',
  'heavy intensity rain',
  'NaN',
  'broken clouds'],
 'current': [None,
  None,
  None,
  None,
  None,
  None,
  'NaN',
  None,
  None,
  'NaN',
  None,
  None,
  None,
  None,
  'NaN',
  None]}

In [140]:
weather_pitch = pd.DataFrame.from_dict(weather_pitch)
weather_pitch.drop_duplicates()

ValueError: All arrays must be of the same length

In [161]:
events = {'data': events}
events = pd.DataFrame.from_dict(events['data'])
events.to_csv(f'data/events/events_.csv')
fixtures = pd.DataFrame.from_dict(fixtures)
fixtures.to_csv(f'data/fixtures/fixtures_.csv') 
weather_pitch = pd.DataFrame.from_dict(weather_pitch)
weather_pitch.to_csv(f'data/weather/weather.csv')